<a href="https://colab.research.google.com/github/PHANI-27/ML-PROJECTS/blob/main/Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
import torch
from PIL import Image
import numpy as np

# Load processor and model
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-ade-semantic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-ade-semantic")

# Create a color palette
def create_color_palette(num_classes):
    np.random.seed(42)
    return {i: tuple(np.random.randint(0, 256, 3)) for i in range(num_classes)}

palette = create_color_palette(model.config.num_labels)

# Convert mask to color image
def segmentation_to_color(mask):
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for label_id, color in palette.items():
        color_mask[mask == label_id] = color
    return Image.fromarray(color_mask)

# Segmentation function
def segment_image(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    # Get segmentation logits and get the most likely class for each pixel
    logits = outputs.logits  # [1, num_classes, H, W]
    predicted = torch.argmax(logits, dim=1)[0].cpu().numpy()  # [H, W]

    color_seg = segmentation_to_color(predicted)
    return color_seg

# Gradio interface
gr.Interface(
    fn=segment_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="pil"),
    title="Semantic Segmentation with Mask2Former (ADE20K)",
    description="Upload an image to segment it using Facebook's Mask2Former model trained on ADE20K dataset."
).launch()
